In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # 1채널 입력을 32채널로 변환하는 3x3 컨볼루션 레이어
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # 32채널 입력을 64채널로 변환하는 3x3 컨볼루션 레이어
        self.dropout1 = nn.Dropout(0.25)  # 25%의 드롭아웃 비율을 가지는 드롭아웃 레이어
        self.dropout2 = nn.Dropout(0.5)  # 50%의 드롭아웃 비율을 가지는 드롭아웃 레이어
        self.fc1 = nn.Linear(9216, 128)  # 9216개의 입력을 128개의 출력으로 변환하는 fully connected 레이어
        self.fc2 = nn.Linear(128, 10)  # 128개의 입력을 10개의 출력으로 변환하는 fully connected 레이어

    def forward(self, x):
        x = self.conv1(x)  # 첫 번째 컨볼루션 레이어를 통과한 결과
        x = F.relu(x)  # ReLU 활성화 함수를 적용한 결과
        x = self.conv2(x)  # 두 번째 컨볼루션 레이어를 통과한 결과
        x = F.relu(x)  # ReLU 활성화 함수를 적용한 결과
        x = F.max_pool2d(x, 2)  # 2x2 최대 풀링 레이어를 통과한 결과
        x = self.dropout1(x)  # 첫 번째 드롭아웃 레이어를 통과한 결과
        x = torch.flatten(x, 1)  # 1차원으로 펼친 결과
        x = self.fc1(x)  # 첫 번째 fully connected 레이어를 통과한 결과
        x = F.relu(x)  # ReLU 활성화 함수를 적용한 결과
        x = self.dropout2(x)  # 두 번째 드롭아웃 레이어를 통과한 결과
        x = self.fc2(x)  # 두 번째 fully connected 레이어를 통과한 결과
        output = F.log_softmax(x, dim=1)  # 로그-소프트맥스 활성화 함수를 적용한 결과
        return output

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 데이터와 타겟을 디바이스로 이동
        optimizer.zero_grad()  # 옵티마이저의 그래디언트 초기화
        output = model(data)  # 모델에 데이터를 입력하여 출력값 계산
        loss = F.nll_loss(output, target)  # 출력값과 타겟을 비교하여 손실 계산
        loss.backward()  # 손실에 대한 그래디언트 계산
        optimizer.step()  # 옵티마이저를 사용하여 모델 파라미터 업데이트
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))  # 현재 배치의 학습 진행 상황 출력
            if args.dry_run:
                break  # dry_run이 True인 경우 학습을 한 번만 수행하도록 중단

In [ ]:
def test(model, device, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    test_loss = 0  # 테스트 손실 초기화
    correct = 0  # 정확한 예측 수 초기화
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for data, target in test_loader:  # 테스트 데이터로더에서 데이터와 타겟 가져오기
            data, target = data.to(device), target.to(device)  # 데이터와 타겟을 디바이스로 이동
            output = model(data)  # 모델에 데이터 입력하여 출력 얻기
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # 배치 손실 합산
            pred = output.argmax(dim=1, keepdim=True)  # 최대 로그 확률의 인덱스 가져오기
            correct += pred.eq(target.view_as(pred)).sum().item()  # 정확한 예측 수 합산

    test_loss /= len(test_loader.dataset)  # 평균 테스트 손실 계산

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))  # 테스트 결과 출력


In [ ]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')  # PyTorch MNIST 예제에 대한 설명을 포함하는 인수 파서 생성
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')  # 훈련에 사용할 입력 배치 크기를 설정하는 인수 추가
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')  # 테스트에 사용할 입력 배치 크기를 설정하는 인수 추가
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')  # 훈련할 에포크 수를 설정하는 인수 추가
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')  # 학습률을 설정하는 인수 추가
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')  # 학습률 스텝 감마를 설정하는 인수 추가
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')  # CUDA 훈련을 비활성화하는 인수 추가
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')  # macOS GPU 훈련을 비활성화하는 인수 추가
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')  # 단일 패스를 빠르게 확인하는 인수 추가
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')  # 랜덤 시드를 설정하는 인수 추가
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')  # 훈련 상태를 기록하기 전에 기다려야 할 배치 수를 설정하는 인수 추가
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')  # 현재 모델을 저장하는 인수 추가
    args = parser.parse_args()  # 명령행 인수를 파싱하여 args에 저장
    use_cuda = not args.no_cuda and torch.cuda.is_available()  # CUDA를 사용할 수 있는지 확인하여 use_cuda에 저장
    use_mps = not args.no_mps and torch.backends.mps.is_available()  # MPS를 사용할 수 있는지 확인하여 use_mps에 저장

    torch.manual_seed(args.seed)  # 랜덤 시드 설정

    if use_cuda:  # CUDA를 사용할 수 있는 경우
        device = torch.device("cuda")  # device를 CUDA로 설정
    elif use_mps:  # MPS를 사용할 수 있는 경우
        device = torch.device("mps")  # device를 MPS로 설정
    else:  # CUDA 및 MPS를 사용할 수 없는 경우
        device = torch.device("cpu")  # device를 CPU로 설정

    train_kwargs = {'batch_size': args.batch_size}  # 훈련 데이터로더에 대한 인수 설정
    test_kwargs = {'batch_size': args.test_batch_size}  # 테스트 데이터로더에 대한 인수 설정
    if use_cuda:  # CUDA를 사용할 수 있는 경우
        cuda_kwargs = {'num_workers': 1, # 사용 cpu 대수
                       'pin_memory': True, # 메모리 잡기술로 속도 업
                       'shuffle': True}  # epoch마다 데이터 섞을지 여부
        train_kwargs.update(cuda_kwargs)  # 훈련 데이터로더에 CUDA 관련 인수 추가
        test_kwargs.update(cuda_kwargs)  # 테스트 데이터로더에 CUDA 관련 인수 추가



 
    transform=transforms.Compose([  # 데이터 변환을 위한 Compose 객체 생성
        transforms.ToTensor(),  # 이미지를 텐서로 변환하는 변환 추가
        transforms.Normalize((0.1307,), (0.3081,))  # 이미지를 정규화하는 변환 추가 (평균, 표준편차 값)
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)  # 훈련 데이터셋 생성
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)  # 테스트 데이터셋 생성
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)  # 훈련 데이터로더 생성
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)  # 테스트 데이터로더 생성




    model = Net().to(device)  # 모델 생성 및 device에 할당
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)  # 옵티마이저 생성



    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)  # 스케줄러 생성
    for epoch in range(1, args.epochs + 1):  # 에포크 반복
        train(args, model, device, train_loader, optimizer, epoch)  # 훈련 함수 호출
        test(model, device, test_loader)  # 테스트 함수 호출
        scheduler.step()  # 스케줄러 갱신

    if args.save_model:  # 모델 저장이 설정된 경우
        torch.save(model.state_dict(), "mnist_cnn.pt")  # 모델 저장